## Simulation Tests

Fixed conditions comprise of sampling time (1600 timesteps to reflect 20 minute flight in seconds), survey area (7500m x 7500m) and sensor detection zone (based on camera model). Changing variables per flight will reflect survey effort and animal availability:

1.Survey Effort

* UAV path (Lawn-mower/ Figure 8)
* FOV (BIOT/ Maldives/ Belize)
* UAV speed (lowest speed-average/ average-max speed)

2.Animal availability

* Animal movement model (Straight Line/ Stop-Start/ Random Walk)
* Animal speed (different per taxa)
* Availability bias (None/Sharks/Rays/Birds)


### Speed  Functions

* Establishing seperate speed profiles

In [1]:
from scipy.stats import truncnorm
import scipy.stats as stats
import matplotlib.pyplot as plt

def get_truncated_normal(mean=0, sd=1, low=0, upp=10):
    # More intuitive method to use truncnorm for speed_profile distribution

    return truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)



def speed_profile (level = 0, mean = 18, sd = 1, low = 4, upp = 33):
    # Speed profile for random sample distribution
    # Average speed: 18 m/s 
    # Level 1 = low, Level 2 = high
    
    speed_dist = get_truncated_normal(mean, sd, low, upp)
    value = speed_dist.rvs(size=1) # Select 1 random number from this distribution
    
    if level == 1:
        while value > mean:
            value = speed_dist.rvs(size=1)


    elif level == 2:
        while value < mean:
            value = speed_dist.rvs(size=1)

    else:
        value = ('You have not selected correct uav level, must be 1 or 2', 0)
    
    return value[0]
        

# Show graphics
#lower, upper = 4, 33
#mu, sigma = 18, 2
#X = stats.truncnorm(
#    (lower - mu) / sigma, (upper - mu) / sigma, loc=mu, scale=sigma)
#N = stats.norm(loc=mu, scale=sigma)

#fig, ax = plt.subplots(2, sharex=True)
#ax[0].hist(X.rvs(10000), normed=True) # truncated normal dist
#ax[1].hist(N.rvs(10000), normed=True) # normal dist
#plt.show()

speed_profile(2)

20.608263589363496


### Speed profiles of animal

#### Elasmos & Mammals
1.  Grey reef shark (Ryan, 15)
Cruise speed - 0.64m/s | max cruise speed - 1.23 | sd - 1.23-0.35/4

2. Nurse shark (Whitney, 16)
Cruise speed is 0.33Bl/s - 0.37m/s (Mean of the mean swimming speeds in 21 - 30 degree water) ((((91+132)/2)/100)*0.33) | max - ((132/100)*0.42) (Mximum speed range with maximum body size). | sd -  (0.59 - ((91/100)*0.23))/ 4 

3. Whale shark (Jacoby,18)
Cruise speed is 0.6m/s | max - 1.06 | 1.06-0.09/4 

4. Mantaray (using for eagleray) (Fish, 18)  
Cruise speed is 1.42 m/s | max - 2.51 | 2.51-0.46/4

5. Manatee speed (Kojeszewski, 07)
Cruise speed is 0.7m/s | max - 1.14 | 1.14-0.06/4

#### Birds
1. Red-footed boobies (Weimerskirch, 05)
Median speed of 38km/h - 10.5m/s | max - 22m/s | 22-2.7/4

2. Frigatebird (Weimerskirch,03)
Average speed is 16.4km/h - 4.5 m/s | max - 5.1m/s | 5.1-2.7/4

3. Common tern average (Hedenstrom,16)
Average is 8.5 m/s | max - 10m/s | 10-7/4

#### Calculating standard deviation

Rough estimate of standard deviation derived from the range rule for standard deviation;

$$\frac {Maximum - Minimum}{4} $$

Why 4? 95% of the data will fall within 2 standard deveiations either side of the mean and so nearly all of our normal distribution will stretch over 4 standard deviations (Wan, 14).


In [51]:
from matplotlib import pyplot as plt
import statistics
import math


def animal_speed (animal):
    # Speed distribution is based on crude figures for each animal taxa
    
    speed_dict = {"reef": [0.35, 0.64, 1.23, 0.22], "nurse": [0.21, 0.37, 0.59, 0.1], "whale_shark" : [0.09, 0.6, 1.06, 0.24], "ray": [0.046, 1.42, 2.51, 0.5], 
                  "manatee": [0.06, 0.7, 1.14, 0.27], "booby": [2.7, 10.5, 22, 4.82], "frigate": [2.7, 4.5, 5.1, 0.6], "tern": [7, 8.5, 10, 0.75]}
    
    try:
        x =  speed_dict[animal.lower()]
        mean = x[1]
        upp = x[2]
        sd = x[3]
        low = x[0]
        speed_dist = get_truncated_normal(mean, sd, low, upp)
        value = speed_dist.rvs(size=1)
        return value[0]
    
    except KeyError:
        raise KeyError ('Invalid animal species, please try again')
                        
    #fig, ax = plt.subplots(1, sharex=True)
    #ax.hist(speed_dist.rvs(1000), normed=True)
    #ax.set_title(animal)
    #plt.show()
    



In [52]:
animal_speed('booby')

6.8612847695696475

### UAV and Animal movement functions

1. **find_sample_points** to retrieve coordinates between two points - suitable for set UAV path and Animal straight/stop-start path. Speed level chosen according to individual and sampled randomly while angle remains consistent with direction of movement. No timsteps specified as speed of drone will dictate the number of steps between two points, i.e if the individual moves faster, it will complete the path in a shorter amount of time. Boundary points for the UAV will come into play in the path function.

2. **random_walk** runs for a specified number of timesteps for different animals (with varying speed profiles) within a specified interval of angle ranges. To prevent the individual from moving outside of the boundary, the x, y and then both x and y coordinates are inversed, changing the direction of the individual.

In [53]:
import math
import numpy as np


def round_up(n, decimals=2):
    multiplier = 10 ** decimals
    return math.ceil(n * multiplier) / multiplier

def get_speed(individual, level = 0):
    
    # Speed retrieved depending on which object we are simulating
    # individual = string of uav or animal

    if individual == 'uav':
        s = speed_profile(level)
    else:
        s = animal_speed(individual)
        
    return s


def find_sample_points(start_coord, end_coord, angle, individual, speed_level = 0, probability = 0):

    # Retrieves coordinates for x number of timesteps between 2 points
    
    # start_coord = start point of animal or drone
    # end_coord = end point of animal or drone
    # angle = angle of travel in degrees
    # individual = animal or drone?
    # speed_level = low(1) or high(2)
    # probability = probability of animal remaining stationary in that timestep(default = 0)
    
    
    horiz_dist = abs(end_coord[0] - start_coord[0]) # x coordinates
    vert_dist = abs(end_coord[1] - start_coord[1]) # y coordinates
    length = math.sqrt((horiz_dist**2) + (vert_dist**2)) # length in metres between both points
    angle_radians = math.radians(angle)

    
    total_length_covered = 0
    x_list = [start_coord[0]] # list of x values at every step
    y_list = [start_coord[1]] # list of y values at every step
    iteration = 0 # used to cumulatively add the last x, y values to recent

    while True:
        s = get_speed(individual, speed_level) # get our speed for this step
        total_length_covered += s # cumulatively add to total length covered
        #print(total_length_covered)
        if total_length_covered > length:
            break
        
        point_probability = np.random.choice(100,1) # assign probability to coordinate
        #print(point_probability)
        if point_probability > probability:
            delta_x = x_list[iteration] + (math.cos(angle_radians)* s) # From start x coordinate, add the length with direction calculated through cosine 
            delta_y = y_list[iteration] + (math.sin(angle_radians)* s) # From start y coordinate, add the length with direction calculated through sine
          
            x = round_up(delta_x)
            y = round_up(delta_y)
            
            x_list.append(x) 
            y_list.append(y)
        
        else:
            total_length_covered = total_length_covered - s # Take last distance as we have not travelled here
            x_list.append(x_list[iteration]) 
            y_list.append(y_list[iteration])

        iteration += 1


    sample_coords = list(zip(x_list, y_list))     # zip up to create list of tuples, each being a set of coordinates where a sample has been taken
    
    Actual_length = total_length_covered - s
    
    
    return sample_coords 



In [54]:
test = find_sample_points([45,113], [7450,113], 180, 'uav',1)
test

[(45, 113),
 (27.68, 113.0),
 (27.68, 113.0),
 (9.86, 113.0),
 (-7.13, 113.0),
 (-23.47, 113.0),
 (-41.28, 113.0),
 (-59.22, 113.0),
 (-76.56, 113.0),
 (-93.12, 113.0),
 (-110.94, 113.0),
 (-128.79, 113.0),
 (-145.87, 113.0),
 (-162.87, 113.0),
 (-180.71, 113.0),
 (-198.23, 113.0),
 (-215.25, 113.0),
 (-233.2, 113.0),
 (-250.19, 113.0),
 (-266.33, 113.0),
 (-284.04, 113.0),
 (-300.57, 113.0),
 (-317.71, 113.0),
 (-335.23, 113.0),
 (-350.4, 113.0),
 (-366.78, 113.0),
 (-383.87, 113.0),
 (-401.01, 113.0),
 (-418.34, 113.0),
 (-434.5, 113.0),
 (-451.36, 113.0),
 (-467.85, 113.0),
 (-484.58, 113.0),
 (-500.42, 113.0),
 (-518.15, 113.0),
 (-535.45, 113.0),
 (-553.41, 113.0),
 (-571.18, 113.0),
 (-587.09, 113.0),
 (-604.31, 113.0),
 (-620.21, 113.0),
 (-637.13, 113.0),
 (-654.97, 113.0),
 (-672.63, 113.0),
 (-690.16, 113.0),
 (-707.4, 113.0),
 (-725.19, 113.0),
 (-741.33, 113.0),
 (-757.57, 113.0),
 (-775.48, 113.0),
 (-792.9, 113.0),
 (-810.54, 113.0),
 (-828.13, 113.0),
 (-845.29, 113.0),


### UAV explicit paths

In [55]:
def detection_zone(camera):
    # FOV calculated per camera model in Strip_Sampling_Density.
    # [Height, width]
    
    camdict = {"garmin": [83,113], "sony": [113, 75], "nadir": [164, 109]}
    try:
        return camdict[camera.lower()]
    except KeyError:
        raise KeyError ('Invalid camera model used, should be :{}'.format([x for x in camdict.keys()]))

In [56]:
detection_zone('nadir')

[164, 109]

In [57]:
def boundary_coords_uav(camera, boundary_length = 5000):
    
    # Boundary_length (m)
    # x and y coordinates 
    # coord 1 = bottom righthand side (0,0)
    # coord 2 = bottom lefthand side (7500,0)
    # coord 3 = top lefthand side (7500, 7500)
    # coord 4 = top righthand side (0, 7500)
    
    area = detection_zone(camera)
    half_area = [x/2  for x in area]
    boundary = {'coord_1' : [(0 + half_area[0]), (0 + half_area[1])],
                'coord_2' : [((boundary_length) - half_area[0]), (0 + half_area[1])],
                'coord_3' : [((boundary_length) - half_area[0]), ((boundary_length) - half_area[1])],
                'coord_4' : [(0 + half_area[0]), ((boundary_length) - half_area[1])]}
    
    return boundary

t = boundary_coords_uav('garmin')
t

{'coord_1': [41.5, 56.5],
 'coord_2': [4958.5, 56.5],
 'coord_3': [4958.5, 4943.5],
 'coord_4': [41.5, 4943.5]}

In [58]:
def lawn_shift (last_coord, camera):
    
    # Lawnmower shifts up across the virtual environment
    
    detect_area = detection_zone(camera)
    shift = detect_area[1] # Height of camera detection zone
    new_coord = [last_coord[0], last_coord[1] + shift]
    
    return new_coord
        

    

def figure_8 (camera, speed_level=1):
    
    # Figure 8 path simulation which will depend on boundary coords from camera model 
    # Returns vector of coords from simulated flight
    
    # camera is string variable - garmin or sony
    
    bc = boundary_coords_uav(camera)
    
    leg_1 = find_sample_points(bc['coord_1'], bc['coord_2'], 360, 'uav', speed_level)
    leg_2 = find_sample_points(bc['coord_2'], bc['coord_4'], 135, 'uav', speed_level)
    leg_3 = find_sample_points(bc['coord_4'], bc['coord_3'], 360, 'uav', speed_level)
    leg_4 = find_sample_points(bc['coord_3'], bc['coord_1'], 225, 'uav', speed_level)
    
    track = leg_1 + leg_2 + leg_3 + leg_4
    
    return track
    
    

def lawnmower (camera, speed_level = 1, timestep = 1000):
    
    # Figure 8 path simulation which will depend on boundary coords from camera model 
    # Returns vector of coords from simulated flight
    
    bc = boundary_coords_uav(camera)
    
    point1 = bc['coord_1'] # right hand side coord
    x1 = point1[0] # x value of coord that does not change throughout the changong lawn_shifts
    
    point2 = bc['coord_2'] # left hand side coord
    x2 = point2[0] # x value of coord that does not change throughout the changong lawn_shifts
    
    point3 = bc['coord_3']
    y_limit = point3[1]
    #print(y_limit)
    
    y_shifts = [0] # y value at each new shift
    right_list = []
    left_list = []
    
    while True:
        right_coord = [x1, y_shifts[-1]]
        left_coord = [x2, y_shifts[-1]]
        
        right_list.append(right_coord)
        left_list.append(left_coord)
        
        shift_coord = lawn_shift(right_coord, camera) 
        new_y = shift_coord[1]
        #print(new_y)
        
        if new_y <= y_limit:
            y_shifts.append(new_y)
        else: break
    
    right_list.pop(0) # Remove first y value which is 0
    left_list.pop(0)
    
    points = [[(0,0)],[(x1,0)]] # rough starting point for vertical shift coords to be added, two added because of iteration
    shift_points = []

    
    counter = 0
    
    for i in range(0, len(right_list)):
        print(i)
        if i % 2 == 1:
            #print('i is odd')
            degree = 180
            leg = find_sample_points(left_list[i], right_list[i], degree, 'uav', speed_level)
            

        elif i % 2 == 0:
            #print('i is even')
            degree = 360
            leg = find_sample_points(right_list[i], left_list[i], degree, 'uav', speed_level)
           
      
        points.append(leg)
        #print(counter)
        counter += len(leg)
        shift = find_sample_points(points[i+1][-1], leg[0], 90, 'uav', speed_level) # estimate of distance between 
        #print(shift)
        shift_points.append(shift)
            
        if counter >= timestep:
            break
        
        
     

        #print(len(leg))  
    
    #shift_points.append(shift)
        
    points.pop(0) # Remove the dummy coords above
    points.pop(0)
    
    # Now to merge horizontal legs (points) with vertical shifts for each step
    # Note there are gaps in shift up and start of next leg but not by much
    
    coords = []
    
    for i in range(0, len(points)):
        coords.append(shift_points[i])
        coords.append(points[i])
    
    
    coords.pop(0) # Start from first boundary coordinate
    flat_list = []
    for sublist in coords:
        #Flatten list of lists while retaining nested lists, i.e coordinates
        for item in sublist:
            flat_list.append(item)

            
    
    return flat_list
    
t = lawnmower('garmin')
t

0
1
2
3


[(41.5, 113),
 (58.67, 113.0),
 (75.01, 113.0),
 (91.36, 113.0),
 (107.58, 113.0),
 (125.43, 113.0),
 (142.48, 113.0),
 (160.41, 113.0),
 (178.06, 113.0),
 (196.01, 113.0),
 (213.5, 113.0),
 (230.81, 113.0),
 (248.49, 113.0),
 (266.09, 113.0),
 (284.03, 113.0),
 (301.79, 113.0),
 (318.26, 113.0),
 (336.12, 113.0),
 (353.5, 113.0),
 (369.56, 113.0),
 (386.59, 113.0),
 (403.01, 113.0),
 (420.37, 113.0),
 (437.11, 113.0),
 (454.33, 113.0),
 (471.34, 113.0),
 (488.22, 113.0),
 (505.38, 113.0),
 (520.86, 113.0),
 (538.46, 113.0),
 (555.59, 113.0),
 (572.51, 113.0),
 (589.29, 113.0),
 (606.65, 113.0),
 (622.95, 113.0),
 (640.41, 113.0),
 (657.61, 113.0),
 (673.56, 113.0),
 (689.09, 113.0),
 (705.46, 113.0),
 (722.85, 113.0),
 (739.93, 113.0),
 (757.47, 113.0),
 (774.17, 113.0),
 (789.82, 113.0),
 (807.77, 113.0),
 (824.27, 113.0),
 (840.76, 113.0),
 (858.71, 113.0),
 (876.3, 113.0),
 (892.98, 113.0),
 (910.16, 113.0),
 (927.5, 113.0),
 (945.34, 113.0),
 (962.62, 113.0),
 (979.84, 113.0),
 (9

### Animal paths

1. Simple movement, where S is the probability of stopping and the direction of movement is within the interval [-A, A]
 * S = 0, A = 0 (90 degrees)

2. Stop-start movement
 * S = 0.25, A = 0
 * S = 0.5, A = 0
 * S = 0.75, A = 0
 
3. Correlated random walk
 * S = 0, A = 60
 * S = 0, A = 120
 * S = 0, A = 180

In [59]:
def angle_dist(angle, size = 1):
    
    # Sampling from a uniform distribution between a chosen angle range
    
    low = -(angle)
    high = angle
    t = np.random.uniform(low = low, high = high, size = size)
    
    #plt.hist(t, bins=25, color='#75BEBB')
    #plt.xlabel('Angle (degrees)')
    #plt.ylabel('Frequency')
    #plt.xlim([low - 10, high +10])
    #plt.ylim([0, 75])
    #plt.show()
    
    return t


def random_walk (animal, start_coord, angle, timestep = 1000):
    
    # Correlated random walk for a specified animal with a start coordinate, speed profile to sample from and angle to bound uniform dist
    # x, y and then both x and y coordinates are inversed to ensure point remains within the boundary
    
    x_list = [start_coord[0]] # list of x values at every step
    y_list = [start_coord[1]] # list of y values at every step
    iteration = 0 # used to cumulatively add the last x, y values to recent
    
    
    for i in range(timestep):

            # Select angle and convert to radians
            angle_value = angle_dist(angle)
            angle_radians = math.radians(angle_value)
        
            # Select speed
            s = animal_speed(animal)
        
        
            delta_x = x_list[iteration] + (math.cos(angle_radians)* s) # From start x coordinate, add the length with direction calculated through cosine 
            delta_y = y_list[iteration] + (math.sin(angle_radians)* s) # From start y coordinate, add the length with direction calculated through sine
       
            coords = [delta_x, delta_y]
            
            test = all(i > 0 and i < 7500 for i in coords) # If out of bounds = False 
                
            if test == True: 
                #print('Correct first go:', coords)
                x = round_up(delta_x)
                y = round_up(delta_y)
            
                x_list.append(x) 
                y_list.append(y)
                
                iteration += 1
                
            else: 
                # If out of bounds reverse the x, then the y and then both x and y until you are back within the bounds
                #print('Coords before change:', coords)
                counter = 0
                while test == False:
                    counter = counter + 1
                    if counter == 1:
                        new_x = x_list[iteration] - (math.cos(angle_radians)* s)
                        coords = [new_x, delta_y]
                        #print('Coords first change:', coords)
                        test = all(i > 0 and i < 7500 for i in coords)
                    if counter == 2:
                        new_y = y_list[iteration] - (math.sin(angle_radians)* s)
                        coords = [delta_x, new_y]
                        #print('Coords second change:', coords)
                        test = all(i > 0 and i < 7500 for i in coords)
                    if counter == 3:
                        coords = [new_x, new_y]
                        #print('Coords third change:', coords)
                        test = all(i > 0 and i < 7500 for i in coords)
                        
                    
                    #print('coords passed and being appended:', coords)
                    x = round_up(coords[0])
                    y = round_up(coords[1])
            
                    x_list.append(x) 
                    y_list.append(y)
                
                    iteration += 1
                
                
     
    sample_coords = list(zip(x_list, y_list)) # zip up to create list of tuples, each being a set of coordinates where a sample has been taken
    
    return sample_coords 

random_walk('reef', [1,1], 60)

[(1, 1),
 (1.48, 1.03),
 (2.16, 1.0),
 (2.5, 1.49),
 (2.94, 1.92),
 (3.34, 1.68),
 (3.76, 2.17),
 (4.26, 2.41),
 (4.99, 2.79),
 (5.43, 2.81),
 (6.22, 2.9),
 (6.88, 2.59),
 (7.71, 2.53),
 (8.18, 1.79),
 (8.75, 1.77),
 (9.62, 1.8),
 (10.1, 1.26),
 (10.57, 1.44),
 (11.46, 1.28),
 (12.01, 1.29),
 (12.37, 0.93),
 (13.51, 1.16),
 (14.18, 1.29),
 (14.74, 1.86),
 (15.39, 2.23),
 (16.17, 2.12),
 (16.66, 2.18),
 (17.43, 2.47),
 (18.36, 2.4),
 (18.88, 2.24),
 (19.25, 2.81),
 (19.88, 3.22),
 (20.18, 3.56),
 (20.9, 3.11),
 (21.52, 3.7),
 (22.18, 4.07),
 (22.63, 4.71),
 (23.06, 5.09),
 (23.73, 5.42),
 (24.57, 5.36),
 (25.04, 5.82),
 (25.7, 5.86),
 (26.13, 6.41),
 (26.64, 6.15),
 (27.34, 6.85),
 (27.76, 7.18),
 (28.69, 7.33),
 (29.07, 7.69),
 (30.11, 7.5),
 (30.54, 6.87),
 (31.07, 6.42),
 (31.93, 6.83),
 (32.44, 6.7),
 (33.28, 5.95),
 (33.61, 5.59),
 (34.2, 5.47),
 (35.24, 4.89),
 (35.96, 4.57),
 (36.89, 4.65),
 (37.66, 5.36),
 (38.33, 5.38),
 (39.1, 5.7),
 (39.71, 5.28),
 (40.58, 5.57),
 (41.12, 5.2

## Simulation Runs

* Run UAV and animal path combinations - categorize your runs here for later comparison
* Detection function to mark a hit and at what timestep
* Output table 

In [62]:
def perception_bias(animal):
    # Animal availability - proportion of time spent at surface
    # All obtained from http://www.penguiness.net/thebook (Laran,17)
    # Manatee: Hagihara,14
    
    biasdict = {"reef": 0.1, "nurse": 0.1, "whale_shark" : 0.6, "ray": 0.5, 
                  "manatee": 0.7, "booby": 1, "frigate": 1, "tern": 1}
    try:
        return biasdict[animal.lower()]
    except KeyError:
        raise KeyError ('Invalid animal used, should be :{}'.format([x for x in biasdict.keys()]))




def point_distance (pt1, pt2):
    # Trigonometric distance between 2 points
    
    horiz_dist = abs(pt2[0] - pt1[0]) # x coordinates
    vert_dist = abs(pt2[1] - pt1[1]) # y coordinates
    length = math.sqrt((horiz_dist**2) + (vert_dist**2)) # length in metres between both points
    
    return length


def capture (animal_list, uav_list, camera, timestep = 1000):
    # Return a list of total hits and at what timestep (list of lists)
    # Capture occurs if point lies within boundary so dist between animal and uav must be < half height and half distance
    # Directly on the boarder not captured (probably wouldn't see in an actual image)
    
    # animal_list: list of coordinates
    # uav_list: list of coordinates
    # camera: camera to specify boundary - MUST BE SAME AS UAV 
    # bias: animal perception bias
    # timestep: length of simulation
    
    # Reduce list down to certain number of timesteps (remeber there will be an extra one)
    list1 = animal_list[0:(timestep)]
    list2 = uav_list[0:(timestep)]
    
    # Specify boundary
    area = detection_zone(camera)
    half_area = [x/2  for x in area]
    
    # Hit dict
    hits = []
    
    for i in range(0, len(list1)):
        dist = point_distance(list1[i], list2[i])
        if dist < half_area[0] or dist < half_area[1]:
            #print('Hit at point', list1[i])
            #print(dist)
            #hits[i] = 1
            success = (i,1)
            hits.append(success)
      
    
    return hits



list1 = find_sample_points([3750,0], [3750,7500], 90, 'nurse', 0, 0)
list2 = figure_8('garmin')
#camera = (10,10)
test2 = capture(list1, list2, 'garmin', timestep = 800)
test2

[(213, 1), (214, 1), (215, 1), (216, 1), (217, 1), (218, 1)]

In [67]:
#a = int(round(len(test)*perception_bias('reef')))
#test = test[:-(a)]
#test
#a = [(0,0), (5,6)]
#b = [(9,7), (4,6)]
#t = [point_distance(x,y) for x in a for y in b]
#c = point_distance((0,0), (9,7))


[11.40175425099138, 7.211102550927978, 4.123105625617661, 1.0]

In [63]:
def simulation_animal(animal, path, timestep = 1000):
    # Output vector from simulation
    # S : probability of stopping
    # A: angle
    
    # Straight
    
    
    if path == 'straight':
        vector = find_sample_points([3750,0], [3750,7500], 90, animal, 0, 0)
    
    if path == 'stop25':
        vector = find_sample_points([3750,0], [3750,7500], 90, animal, 0, 0.25)
    
    if path == 'stop50':
        vector = find_sample_points([3750,0], [3750,7500], 90, animal, 0, 0.50)
    
    if path == 'stop75':
        vector = find_sample_points([3750,0], [3750,7500], 90, animal, 0, 0.75)
    
    if path == 'random60':
         vector = random_walk(animal, [1,1], 60, timestep)
            
    if path == 'random120':
        vector = random_walk(animal, [1,1], 120, timestep)
        
    if path == 'random180':
        vector = random_walk(animal, [1,1], 180, timestep)

        
    return vector


simulation_animal('reef', 'straight')

def simulation_uav(camera, path, speed):
    # Output vector for simulation
    
    if path == 'lawnmower':
        vector = lawnmower(camera, speed)
        
    if path == 'figure8':
        vector = figure_8(camera, speed)
        
    return vector
    

In [64]:
import time

def hit_count(vect):
    
    total = []
    hits = []
    
    for i in vect:
        total.append(i[1])
        hits.append(i[0])
    
    total = sum(total)
    
    hit1 = hits[:-1] # Remove last
    hit2 = hits[1:] # Remove first
    
    count = 1
    
    for i in range(0, len(hit2)):
        if(hit2[i] - hit1[i] == 1):
            count = 1 # Remains as 1 if timesteps are ensuing
        else: 
            count += 1 # Add to count score to signifiy new individual
    
    sim_score = [total, count]
    
    return sim_score



def simulation (animal, animal_path, camera, uav_path, speed=1, bias = 'no'):
    # Running the simulation to produce the number of hits
    
    animal_vect = simulation_animal(animal, animal_path)
    uav_vect = simulation_uav(camera, uav_path, speed)
    
    hit_vect = capture(animal_vect, uav_vect, camera)
    
    print('Hits before', len(hit_vect))
    
    if bias == 'yes':
        no_remove = int(round(len(hit_vect)*perception_bias(animal)))
        hits = hit_vect[:-(no_remove)]
    else : pass
    
    print('Hits after', hit_vect)
    
    if len(hit_vect) > 0:
        score = hit_count(hit_vect)
        print(score)
    else:
        score = [0,0]
        
        
    #output = [uav_path, camera, speed, animal, animal_path, bias, score[0], score[1]]
    
    return score

start_time = time.time()
t = simulation('reef', 'straight', 'garmin', 'lawnmower')
print("--- %s seconds ---" % (time.time() - start_time))

t

0
1
2
3
Hits before 12
Hits after [(217, 1), (218, 1), (219, 1), (220, 1), (221, 1), (364, 1), (365, 1), (366, 1), (367, 1), (368, 1), (369, 1), (370, 1)]
[12, 1]
--- 9.315423965454102 seconds ---


[12, 1]